In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
df2 = pd.read_csv("IMDB Dataset.csv")
df2.info()
display(df2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df2.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [4]:
y = df2['sentiment']

In [5]:
y = pd.get_dummies(y)
y

,negative,positive
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
49995,0,1
49996,1,0
49997,1,0
49998,1,0


In [6]:
df2['negative'] = y['negative']
df2['positive'] = y['positive']

In [7]:
df1 = df2.drop(["sentiment"],axis=1)

In [8]:
df1.head()

,review,negative,positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production. <br /><br />The...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically there's a family where a little boy ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1


## Import packages

In [9]:
import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#pip install autocorrect
from autocorrect import Speller
import string
import timeit
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to C:\Users\Agha
[nltk_data]     Zohaib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import unidecode
import nltk

In [11]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatize

[nltk_data] Downloading package stopwords to C:\Users\Agha
[nltk_data]     Zohaib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from nltk.stem import WordNetLemmatizer

In [10]:
!pip install autocorrect

     -------------------------------------- 622.8/622.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=ee3b6bc235e0553a5d7188fb445c6d353732d00d074ac798cd66fc1be203ef15
  Stored in directory: c:\users\agha zohaib\appdata\local\pip\cache\wheels\ab\0f\23\3c010c3fd877b962146e7765f9e9b08026cac8b035094c5750
Successfully built autocorrect


In [13]:
from autocorrect import Speller
import string
import timeit
from bs4 import BeautifulSoup

In [14]:
def remove_newlines_tabs(text):
    """
    This function will remove all the occurrences of newlines, tabs, and combinations like: \\n, \\.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of newlines, tabs, \\n, \\ characters.
        
    Example:
    Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
    Output : This is her first day at this place. Please, Be nice to her. 
    
    """
    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text

remove_newlines_tabs("This is her \\ first day at this place.\n Please,\t Be nice to her.\\n")

'This is her   first day at this place.  Please,  Be nice to her. '

In [15]:
def strip_html_tags(text):
    """ 
    This function will remove all the occurrences of html tags from the text.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of html tags.
        
    Example:
    Input : This is a nice place to live. 
    Output : This is a nice place to live.  
    """
    # Initiating BeautifulSoup object soup.
    soup = BeautifulSoup(text, "html.parser")
    # Get all the text other than html tags.
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


strip_html_tags(" This is a <b>nice</b> place to live. ")

' This is a  nice  place to live. '

In [16]:
def remove_links(text):
    """
    This function will remove all the occurrences of links.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of all types of links.
        
    Example:
    Input : To know more about cats and food & website: catster.com  visit: https://catster.com//how-to-feed-cats

    Output : To know more about cats and food & website: visit:     
    
    """
    
    # Removing all the occurrences of links that starts with https
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com

remove_links(" website: catster.com  visit: https://catster.com//how-to-feed-cats")

' website:   visit: '

In [17]:
def remove_whitespace(text):
    """ This function will remove 
        extra whitespaces from the text
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after extra whitespaces removed .
        
    Example:
    Input : How   are   you   doing   ?
    Output : How are you doing ?     
        
    """
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text    

remove_whitespace("How   are  \t you \n   doing? (pakistan)ABC")

'How are you doing ?  (pakistan) ABC'

In [18]:
# Code for accented characters removal
def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text

accented_characters_removal("Málaga, àéêöhello")

'Malaga, aeeohello'

In [19]:
# Code for text lowercasing
def lower_casing_text(text):
    
    """
    The function will convert text into lower case.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text in lowercase
         
    Example:
    Input : The World is Full of Surprises!
    Output : the world is full of surprises!
    
    """
    # Convert text to lower case
    # lower() - It converts all upperase letter of given string to lowercase.
    text = text.lower()
    return text

lower_casing_text("Pakistan Zinda BAD!")

'pakistan zinda bad!'

In [20]:
def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

reducing_incorrect_character_repeatation("Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)")

'Reallyy, Greeaatt !?.;:)'

In [21]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}
# The code for expanding contraction words
def expand_contractions(text, contraction_mapping =  CONTRACTION_MAP):
    """expand shortened words to the actual form.
       e.g. don't to do not
    
       arguments:
            input_text: "text" of type "String".
         
       return:
            value: Text with expanded form of shorthened words.
        
       Example: 
       Input : ain't, aren't, can't, cause, can't've
       Output :  is not, are not, cannot, because, cannot have 
    
     """
    # Tokenizing text into tokens.
    list_Of_tokens = text.split(' ')

    # Checking for whether the given token matches with the Key & replacing word with key's value.
    
    # Check whether Word is in lidt_Of_tokens or not.
    for Word in list_Of_tokens: 
        # Check whether found word is in dictionary "Contraction Map" or not as a key. 
         if Word in CONTRACTION_MAP: 
                # If Word is present in both dictionary & list_Of_tokens, replace that word with the key value.
                list_Of_tokens = [item.replace(Word, CONTRACTION_MAP[Word]) for item in list_Of_tokens]
                
    # Converting list of tokens to String.
    String_Of_tokens = ' '.join(str(e) for e in list_Of_tokens) 
    return String_Of_tokens    

expand_contractions("ain't , aren't , can't , cause , can't've")
   

"is not , are not , cannot , cause , cannot've"

In [29]:
# The code for removing special characters
def removing_special_characters(text):
    """Removing all the special characters except the one that is passed within 
       the regex to match, as they have imp meaning in the text provided.
   
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text with removed special characters that don't require.
        
    Example: 
    Input : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
    Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
   """
    # The formatted text after removing not necessary punctuations.
    Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text) 
    # In the above regex expression,I am providing necessary set of punctuations that are frequent in this particular dataset.
    return Formatted_Text

removing_special_characters(" Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) ")

' Hello, K a j a l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) '

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Agha
[nltk_data]     Zohaib\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
# The code for removing stopwords
stoplist = stopwords.words('english') 

stoplist = set(stoplist)
def removing_stopwords(text):
    """This function will remove stopwords which doesn't add much meaning to a sentence 
       & they can be remove safely without comprimising meaning of the sentence.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after omitted all stopwords.
        
    Example: 
    Input : This is Kajal from delhi who came here to study.
    Output : ["'This", 'Kajal', 'delhi', 'came', 'study', '.', "'"] 
    
   """
    # repr() function actually gives the precise information about the string
    text = repr(text)
    # Text without stopwords
    No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
    # Convert list of tokens_without_stopwords to String type.
    words_string = ' '.join(No_StopWords)    
    return words_string

removing_stopwords("This is Kajal from the delhi who was came here to study.")

"'This Kajal delhi came study . '"

In [24]:
# The code for spelling corrections
def spelling_correction(text):
    ''' 
    This function will correct spellings.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after corrected spellings.
        
    Example: 
    Input : This is Oberois from Dlhi who came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.
      
    
    '''
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text

spelling_correction("This is Oberois from Dlhi who came heree to studdy.")

'This is Oberoi from Delhi who came here to study.'

In [25]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\Agha
[nltk_data]     Zohaib\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Agha
[nltk_data]     Zohaib\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [26]:
# The code for lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatization(text):
    """This function converts word to their root words 
       without explicitely cut down as done in stemming.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text having root words only, no tense form, no plural forms
        
    Example: 
    Input : text reduced 
    Output :  text reduce
    
   """
    # Converting words to their root forms
    lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
    return lemma

lemmatization("ext having root words only, no tense form, no plural forms cats dogs")

['ext',
 'have',
 'root',
 'word',
 'only,',
 'no',
 'tense',
 'form,',
 'no',
 'plural',
 'form',
 'cat',
 'dog']

In [27]:
lemmatizer.lemmatize('educations','v')

'educations'

In [30]:
def text_preprocessing(text, accented_chars=True, contractions=True, lemmatization_allow = True,
                        extra_whitespace=True, newlines_tabs=True, repeatition=True, 
                       lowercase=True, punctuations=True, mis_spell=True,
                       remove_html=True, links=True,  special_chars=True,
                       stop_words=True):
    """
    This function will preprocess input text and return
    the clean text.
    """
    
    if newlines_tabs == True: #remove newlines & tabs.
        Data = remove_newlines_tabs(text)
        
    if remove_html == True: #remove html tags
        Data = strip_html_tags(Data)
        
    if links == True: #remove links
        Data = remove_links(Data)
        
    if extra_whitespace == True: #remove extra whitespaces
        Data = remove_whitespace(Data)
        
    if accented_chars == True: #remove accented characters
        Data = accented_characters_removal(Data)
        
    if lowercase == True: #convert all characters to lowercase
        Data = lower_casing_text(Data)
        
    if repeatition == True: #Reduce repeatitions   
        Data = reducing_incorrect_character_repeatation(Data)
        
    if contractions == True: #expand contractions
        Data = expand_contractions(Data)
    
    if punctuations == True: #remove punctuations
        Data = removing_special_characters(Data)
    
    stoplist = stopwords.words('english') 
    stoplist = set(stoplist)
    
    if stop_words == True: #Remove stopwords
        Data = removing_stopwords(Data)
        
    spell = Speller(lang='en')
    
    if mis_spell == True: #Check for mis-spelled words & correct them.
        Data = spelling_correction(Data)
        
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
#     print(Data)
    if lemmatization_allow == True: #Converts words to lemma form.
        Data = lemmatization(Data)
    
           
    return Data
text_preprocessing(' would have a hard time sitting through this one dogs ')

["'", 'would', 'hard', 'time', 'sit', 'one', 'dog', "'"]

In [31]:
df1.head()

,review,negative,positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production. <br /><br />The...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically there's a family where a little boy ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1


In [33]:
data = df1[0:5000]

In [35]:
%%time
data['input_data'] = data.review.apply(text_preprocessing)
data.head()

C:\Users\Agha Zohaib\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Wall time: 1h 2min 27s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,review,negative,positive,input_data
0,One of the other reviewers has mentioned that ...,0,1,"['one, reviewers, mention, watch, 1, oz, episo..."
1,A wonderful little production. <br /><br />The...,0,1,"[``, wonderful, little, production, ., film, t..."
2,I thought this was a wonderful way to spend ti...,0,1,"[``, think, wonderful, way, spend, time, hot, ..."
3,Basically there's a family where a little boy ...,1,0,"['basically, family, little, boy, (, jake, ), ..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1,"[``, better, matter, 's, love, time, money, vi..."


In [36]:
data.review = data.review.apply(str)
data.head()

C:\Users\Agha Zohaib\AppData\Local\Temp\ipykernel_8444\1162815898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.review = data.review.apply(str)


,review,negative,positive,input_data
0,One of the other reviewers has mentioned that ...,0,1,"['one, reviewers, mention, watch, 1, oz, episo..."
1,A wonderful little production. <br /><br />The...,0,1,"[``, wonderful, little, production, ., film, t..."
2,I thought this was a wonderful way to spend ti...,0,1,"[``, think, wonderful, way, spend, time, hot, ..."
3,Basically there's a family where a little boy ...,1,0,"['basically, family, little, boy, (, jake, ), ..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1,"[``, better, matter, 's, love, time, money, vi..."


In [38]:
data['input_data1'] = data.input_data.apply(lambda x:" ".join(x))
data.head()

C:\Users\Agha Zohaib\AppData\Local\Temp\ipykernel_8444\1282143616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['input_data1'] = data.input_data.apply(lambda x:" ".join(x))


,review,negative,positive,input_data,input_data1
0,One of the other reviewers has mentioned that ...,0,1,"['one, reviewers, mention, watch, 1, oz, episo...",'one reviewers mention watch 1 oz episode hook...
1,A wonderful little production. <br /><br />The...,0,1,"[``, wonderful, little, production, ., film, t...",`` wonderful little production . film techniqu...
2,I thought this was a wonderful way to spend ti...,0,1,"[``, think, wonderful, way, spend, time, hot, ...",`` think wonderful way spend time hot summer w...
3,Basically there's a family where a little boy ...,1,0,"['basically, family, little, boy, (, jake, ), ...",'basically family little boy ( jake ) think zo...
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1,"[``, better, matter, 's, love, time, money, vi...",`` better matter 's love time money visually s...


In [39]:
test = pd.concat([data[data.positive==0].sample(2000),
                    data[data.positive==1].sample(1500)])[["input_data1","positive"]]
print(len(test))
display(test.head(1))

train = data.loc[~data.index.isin(test.index.values)][["input_data1","positive"]]
print(len(train))
display(train.head(1))

3500


,input_data1,positive
391,"`` skip family tie play eddie , pussy 'metal '...",0


1500


,input_data1,positive
0,'one reviewers mention watch 1 oz episode hook...,1


In [40]:
X = data[["input_data1"]]# all Data for corpus

print(X.head())

                                         input_data1
0  'one reviewers mention watch 1 oz episode hook...
1  `` wonderful little production . film techniqu...
2  `` think wonderful way spend time hot summer w...
3  'basically family little boy ( jake ) think zo...
4  `` better matter 's love time money visually s...


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = X.input_data1.values
vectorizer = TfidfVectorizer()
vectorizer.fit(text)
print(vectorizer.vocabulary_)

{'one': 14115, 'reviewers': 16843, 'mention': 12634, 'watch': 21729, 'oz': 14456, 'episode': 6813, 'hook': 9555, 'right': 16941, 'exactly': 7022, 'happen': 9023, 'first': 7632, 'thing': 20047, 'strike': 19200, 'brutality': 2917, 'unflinching': 20925, 'scenes': 17514, 'violence': 21489, 'set': 17866, 'word': 22112, 'go': 8505, 'trust': 20587, 'show': 18078, 'faint': 7271, 'hearted': 9185, 'timid': 20181, 'pull': 15915, 'punch': 15925, 'regard': 16474, 'drug': 6222, 'sex': 17895, 'hardcore': 9044, 'classic': 3924, 'use': 21181, 'call': 3145, 'nickname': 13664, 'give': 8438, 'onward': 14136, 'maximum': 12428, 'security': 17712, 'state': 18986, 'penitentiary': 14827, 'focus': 7778, 'mainly': 12105, 'emerald': 6598, 'city': 3882, 'experimental': 7129, 'section': 17706, 'prison': 15632, 'cells': 3505, 'glass': 8460, 'front': 8049, 'face': 7235, 'inwards': 10565, 'privacy': 15638, 'high': 9353, 'agenda': 813, 'em': 6572, 'home': 9502, 'many': 12238, 'areas': 1460, 'muslims': 13342, 'gangster'

In [42]:
train_x = vectorizer.transform(train.input_data1.values).toarray()
train_y = train.positive.values
test_x = vectorizer.transform(test.input_data1.values).toarray()
test_y = test.positive.values
print(len(train_x),len(train_y))
print(len(test_x),len(test_y))

1500 1500
3500 3500


In [43]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(train_x,train_y)

BernoulliNB()

In [44]:
y_predict = clf.predict(test_x)
y_predict

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(test_y,y_predict)
cm

array([[ 575, 1425],
       [  61, 1439]], dtype=int64)

In [46]:
accuracy_score(test_y, y_predict)

0.5754285714285714

In [47]:
import pickle
filename = 'IMBD_model.sav'
pickle.dump(y_predict, open(filename, 'wb'))